In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [57]:
import struct
from struct import *
#define a function (ReadBMP) which will take input as image filename
def ReadBMP():
#Ask the user to enter the filename
    img = input("Enter the filename: ")
#using the built in python function to open the imagefile: open(file, mode='r'), r=text mode; b=binary mode
    bmp = open(img, 'rb')
    
#(a)BMP MAIN HEADER STRUCTURE 

#The header field used to identfy BMP file. 2 bytes. Decoding is done to convert from one encoding scheme to another 
    print('Type:', bmp.read(2).decode())
#Size of the bmp file in bytes. Buffer size should be the same as the size required by the format (4 bytes)
    print('Size: %s' % struct.unpack('I', bmp.read(4)))
# Two bytes each for 2 reserved spaces.
    print('Reserved 1: %s' % struct.unpack('H', bmp.read(2)))
    print('Reserved 2: %s' % struct.unpack('H', bmp.read(2)))
#The offset, i.e. starting address, of the byte where the bitmap image data (pixel array) can be found.(4 bytes)
    print('The Offset: %s' % struct.unpack('I', bmp.read(4)))
    
#DIB HEADER STRUCTURE

#the size of this header (40 bytes)
    print('DIB Header Size: %s' % struct.unpack('I', bmp.read(4)))
#the bitmap width in pixels (signed integer)
    print('Width: %s' % struct.unpack('I', bmp.read(4)))
#the bitmap height in pixels (signed integer)
    print('Height: %s' % struct.unpack('I', bmp.read(4)))
#the number of color planes (must be 1)
    print('Colour Planes: %s' % struct.unpack('H', bmp.read(2)))
#the number of bits per pixel, which is the color depth of the image. Typical values are 1, 4, 8, 16, 24 and 32.
    print('Bits per Pixel: %s' % struct.unpack('H', bmp.read(2)))
#the compression method being used. 
    print('Compression Method: %s' % struct.unpack('I', bmp.read(4)))
#the image size. This is the size of the raw bitmap data; a dummy 0 can be given for BI_RGB bitmaps.
    print('Raw Image Size: %s' % struct.unpack('I', bmp.read(4)))
#the horizontal resolution of the image. (pixel per metre, signed integer)
    print('Horizontal Resolution: %s' % struct.unpack('I', bmp.read(4)))
#the vertical resolution of the image. (pixel per metre, signed integer)
    print('Vertical Resolution: %s' % struct.unpack('I', bmp.read(4)))
#the number of colors in the color palette, or 0 to default to 2n
    print('Number of Colours: %s' % struct.unpack('I', bmp.read(4)))
#the number of important colors used, or 0 when every color is important; generally ignored
    print('Important Colours: %s' % struct.unpack('I', bmp.read(4)))

#(b)LOADING IMAGE FILE INTO AN ARRAY

# BMP isn't a text file format, it's a binary format. Since a bytes object isn't mutable, 
# we will probably want to copy it into a bytearray to work with  
    mbr = bytearray(bmp.read())
    struct.unpack_from('<L', mbr) 
    new_data = mbr[0:786432]
#separating the data into red, green and blue bytearray and simoultaneously converting to numpy array
    R = np.array(new_data[::3])
    B = np.array(new_data[1::3])
    G = np.array(new_data[2::3])
#formula for converting an RGB matrix to a grayscale form
    gray = 0.2989 * R + 0.5870 * G + 0.1140 * B
#reshaping the 1-D matrix obtained above into 512x512 format
    arr_reshape = np.reshape(gray,(512,512))
#diagonally flipping(transpose) of the image matrix
    transpose = np.transpose(arr_reshape)
    print(normal)
    
#(c)WRITING BMP FILE



In [52]:
#A module for dealing with BMP bitmap image files.


def write_grayscale(filename, pixels):
    #Creates and writes a grayscale BMP file
    #Args:
     #   filename: The name of the BMP file to be crated.
      #  pixels: A rectangular image stored as a sequence of rows.
       #     Each row must be an iterable series of integers in the range 0-255.
    #Raises:
     #   OSError: If the file couldn't be written.
    height = len(pixels)
    width = len(pixels[0])

    with open(abc.bmp, 'wb') as bmp:
        # BMP Header
        bmp.write(b'BM')

        size_bookmark = bmp.tell()  # The next four bytes hold the filesize as a 32-bit
        bmp.write(b'\x00\x00\x00\x00')  # little-endian integer. Zero placeholder for now.

        bmp.write(b'\x00\x00')  # Unused 16-bit integer - should be zero
        bmp.write(b'\x00\x00')  # Unused 16-bit integer - should be zero

        pixel_offset_bookmark = bmp.tell()  # The next four bytes hold the integer offset
        bmp.write(b'\x00\x00\x00\x00')  # to the pixel data. Zero placeholder for now.

        # Image header
        bmp.write(b'\x28\x00\x00\x00')  # Image header size in bytes - 40 decimal
        bmp.write(_int32_to_bytes(width))  # Image width in pixels
        bmp.write(_int32_to_bytes(height))  # Image height in pixels
        bmp.write(b'\x01\x00')  # Number of image planes
        bmp.write(b'\x08\x00')  # Bits per pixel 8 for grayscale
        bmp.write(b'\x00\x00\x00\x00')  # No compression
        bmp.write(b'\x00\x00\x00\x00')  # Zero for uncompressed images
        bmp.write(b'\x00\x00\x00\x00')  # Unused pixels per meter
        bmp.write(b'\x00\x00\x00\x00')  # Unused pixels per meter
        bmp.write(b'\x00\x00\x00\x00')  # Use whole color table
        bmp.write(b'\x00\x00\x00\x00')  # All colors are important

        # Color palette - a linear grayscale
        for c in range(256):
            bmp.write(bytes((c, c, c, 0)))

        # Pixel data
        pixel_data_bookmark = bmp.tell()
        for row in reversed(pixels):  # BMP files are bottom to top
            row_data = bytes(row)
            bmp.write(row_data)
            padding = b'\x00' * ((4 - (len(row) % 4)) % 4)  # Pad row to multiple of four bytes
            bmp.write(padding)

        # End of file
        eof_bookmark = bmp.tell()

        # Fill in file size placeholder
        bmp.seek(size_bookmark)
        bmp.write(_int32_to_bytes(eof_bookmark))

        # Fill in pixel
        bmp.seek(pixel_offset_bookmark)
        bmp.write(_int32_to_bytes(pixel_data_bookmark))


def _int32_to_bytes(i):
    #Convert an integer to four bytes in little-endian format.
    return bytes((i & 0xff,
                  i >> 8 & 0xff,
                  i >> 16 & 0xff,
                  i >> 24 & 0xff))


def dimensions(filename):
    #Determine the dimensions in pixels of a BMP image.
    #Args:
        #filename: The filename of a BMP file.
    #Returns:
        #A tuple containing two integer with the width
        #and height in pixels.
    #Raises:
     #   ValueError: If the file was not aBMP file.
      #  OSError: If there was a problem reading the file.

    with open(LenaRGB.bmp, 'rb') as f:
        magic = f.read(2)
        if magic != b'BM':
            raise ValueError("{} is not a BMP file".format(filename))

        f.seek(18)
        width_bytes = f.read(4)
        height_bytes = f.read(4)

        return (_bytes_to_int32(width_bytes),
                _bytes_to_int32(height_bytes))


def _bytes_to_int32(b):
    return b[0] | (b[1] << 8) | (b[2] << 16 | (b[3] << 24))